# Monowai CC tests

In [28]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from glob import glob
import os
import xcorr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
%matplotlib notebook

### open a file

In [30]:
root = '../data/results'
time = pd.to_datetime('2015-01-15')

hydro = {
    'network'  : 'IM',
    'station'  : 'H10N1',
    'location' : '',
    'channel'  : 'EDH',
}
seism = {
    'network'  : 'IU',
    'station'  : 'RAR',
    'location' : '10',
    'channel'  : 'BHZ',
}

vel = dict( min=1.46, max=1.50 )
filter_params = dict(frequency=3., btype='highpass', order=2, inplace=True)
taper_params = dict(max_length=2/3., inplace=True)

bhz = xcorr.read(xcorr.util.ncfile((hydro, seism), time, root), fast=True)
bhr = xcorr.read(xcorr.util.ncfile((hydro, {**seism,'channel':'BHR'}), time, root), fast=True)

assert bhz, 'No data found!'

if bhz:
    xcorr.signal.filter(bhz.cc, **filter_params)
    xcorr.signal.taper(bhz.cc, **taper_params)
    xcorr.bias_correct(bhz,unbiased_var='cc_w')
if bhr:
    xcorr.signal.filter(bhr.cc, **filter_params)
    xcorr.signal.taper(bhr.cc, **taper_params)
    xcorr.bias_correct(bhr,unbiased_var='cc_w')

### CCF noise window

In [ ]:
plt.figure(figsize=[9,4])
bhz.cc_w.loc[{'time':bhz.time[1]}].plot.line(x='lag',add_legend=False,color='orange')
bhz.cc.loc[{'time':bhz.time[1]}].plot.line(x='lag',add_legend=False,color='blue')

In [ ]:
noise_win = xcorr.signal.mask(
    x=bhz.lag,
    lower=.2,
    upper=.25,
    scalar=bhz.time.window_length
)

signal_win = xcorr.signal.mask(
    x=bhz.lag,
    lower=1/vel['max'],
    upper=1/vel['min'],
    scalar=bhz.distance.values[0]
)

valid_win = xcorr.signal.mask(
    x=bhz.lag,
    upper=.25,
    scalar=bhz.time.window_length
)

i = 1

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc.loc[{'time':bhr.time[i],'lag':bhr.lag[valid_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc.loc[{'time':bhz.time[i],'lag':bhz.lag[valid_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc.loc[{'time':bhr.time[i],'lag':bhr.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc.loc[{'time':bhz.time[i],'lag':bhz.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc_w.loc[{'time':bhr.time[i],'lag':bhr.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc_w.loc[{'time':bhz.time[i],'lag':bhz.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc_w.loc[{'time':bhr.time[i],'lag':bhr.lag[noise_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc_w.loc[{'time':bhz.time[i],'lag':bhz.lag[noise_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
bhz['snr'] = xcorr.signal.snr(x=bhz.cc, signal=signal_win, noise=noise_win)
bhr['snr'] = xcorr.signal.snr(x=bhr.cc, signal=signal_win, noise=noise_win)

In [ ]:
bhz.snr

In [ ]:
plt.figure(figsize=[9,4])
bhr.snr.plot.line(x='time',add_legend=False,color='blue')
bhz.snr.plot.line(x='time',add_legend=False,color='orange')

### xcorr full day (#8)

In [ ]:
snr_min = 5

plt.figure(figsize=[9,4])
if bhr:
    snr_pass = bhr.snr.loc[{'pair':bhr.pair[0]}] >= snr_min
    if any(snr_pass):
        bhr.cc_w.loc[{
            'time':bhr.time[snr_pass],
            'lag':bhr.lag[signal_win],
            'pair':bhr.pair[0]
        }].plot.line(x='lag',add_legend=False,color='blue',alpha=0.3)
if bhz:
    snr_pass = bhz.snr.loc[{'pair':bhz.pair[0]}] >= snr_min
    if any(snr_pass):
        bhz.cc_w.loc[{
            'time':bhz.time[snr_pass],
            'lag':bhz.lag[signal_win],
            'pair':bhz.pair[0]
        }].plot.line(x='lag',add_legend=False,color='orange',alpha=0.3)

## Spectrogram

In [ ]:
psd = xcorr.signal.psd(
    bhz.cc_w.loc[{'lag':bhz.lag[signal_win]}],
    duration = 2.,
    padding_factor = 4,
)

In [ ]:
plt.figure()
psd.loc[{'time':psd.time[4],'pair':psd.pair[0]}].plot.imshow(x='lag')

In [ ]:
psd_mean = psd.dropna(dim='lag').sum(dim='time',skipna=True).mean(dim='pair',skipna=True)

In [ ]:
plt.figure()
psd_mean.plot.imshow(x='lag')